# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import numpy as np
from PIL import Image

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
validation_transforms = transforms.Compose([transforms.Resize(225),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406],
                                                                 [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(225),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406],
                                                                 [0.229, 0.224, 0.225])])
# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(data_dir + '/train', transform = train_transforms)
validation_data = datasets.ImageFolder(data_dir + '/valid', transform = validation_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform = test_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size = 64, shuffle = True)
validationloader = torch.utils.data.DataLoader(validation_data, batch_size = 64)
testloader = torch.utils.data.DataLoader(test_data, batch_size = 64)

# image_datasets is a dictionary which has train, validation and test datasets(with transforms)
image_datasets = {
    'train':datasets.ImageFolder(data_dir + '/train', transform = train_transforms),
    'validation':datasets.ImageFolder(data_dir + '/valid', transform = validation_transforms),
    'test':datasets.ImageFolder(data_dir + '/test', transform = test_transforms)
}

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [5]:
# TODO: Build and train your network
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Step 1 BUILD THE MODEL
model = models.vgg11(pretrained = True)
model


Downloading: "https://download.pytorch.org/models/vgg11-bbd30ac9.pth" to /root/.torch/models/vgg11-bbd30ac9.pth
100%|██████████| 531456000/531456000 [00:06<00:00, 82341641.25it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 512, kern

In [6]:
#The classifier part in VGG model was trained on the ImageNet dataset, 
#so it won't work for our specific problem. That means we need to replace the classifier
#Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout

#Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
# Define new classifer
classifier = nn.Sequential(nn.Linear(25088, 1000),
                           nn.ReLU(),
                           nn.Dropout(p=0.2),
                           nn.Linear(1000, 102),
                           nn.LogSoftmax(dim=1))

# Set the classifier part of the VGG model = classifier
model.classifier = classifier
criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
#Move this model to available device
model.to(device);

#STEP 2 TRAIN THE MODEL
epochs = 1
# Tracking the train steps
steps = 0
# Tracking the loss
running_loss = 0
# How many steps we're going to go before printing out the validation loss
print_every = 5

for epoch in range(epochs):
    # Training loop
    for images, labels in trainloader:
        steps += 1
        
        #move images and labels to GPU
        images, labels = images.to(device), labels.to(device)
        # We need to zero the gradients on each training pass or we'll retain gradients from previous training batches.
        optimizer.zero_grad()
        # Forward pass,  get our logits
        logps = model(images)
        # Calculate the loss with the logits and the labels
        loss = criterion(logps, labels)
        #then backward pass, then update weights
        loss.backward()
        # Take an update step and update the new weights
        optimizer.step()
        
        running_loss += loss.item()
        # STEP 3 Validation loop
        if steps % print_every == 0:
            # turn the model into evaluation mode (turn off dropout)
            model.eval()
            validation_loss = 0
            accuracy = 0
            # Turn off gradients for validation, saves memory and computations
            with torch.no_grad():
                for images, labels in validationloader:
                
                    images, labels = images.to(device), labels.to(device)
                
                    logps = model(images)
                    loss = criterion(logps, labels)
                    validation_loss += loss.item()
                
                    # Calculate the accuracy
                    ps = torch.exp(logps)
                    top_ps, top_class = ps.topk(1, dim=1)
                    equality = top_class == labels.view(*top_class.shape)
                    # Calculate accuracy from equality
                    accuracy += torch.mean(equality.type(torch.FloatTensor))
             
            print(f"Epoch {epoch+1}/{epochs}.."
                  f"Train loss: {running_loss/print_every:.3f}.."
                  f"Validation loss: {validation_loss/len(validationloader): .3f}.."
                  f"Validation accuracy: {accuracy/len(validationloader):.3f}")
            
            running_loss = 0
            model.train()
        

Epoch 1/1..Train loss: 15.898..Validation loss:  14.996..Validation accuracy: 0.110
Epoch 1/1..Train loss: 11.521..Validation loss:  5.703..Validation accuracy: 0.155
Epoch 1/1..Train loss: 5.307..Validation loss:  3.815..Validation accuracy: 0.186
Epoch 1/1..Train loss: 4.017..Validation loss:  3.552..Validation accuracy: 0.268
Epoch 1/1..Train loss: 3.703..Validation loss:  3.346..Validation accuracy: 0.262
Epoch 1/1..Train loss: 3.619..Validation loss:  2.977..Validation accuracy: 0.332
Epoch 1/1..Train loss: 3.286..Validation loss:  2.553..Validation accuracy: 0.380
Epoch 1/1..Train loss: 2.834..Validation loss:  2.358..Validation accuracy: 0.423
Epoch 1/1..Train loss: 3.040..Validation loss:  2.178..Validation accuracy: 0.489
Epoch 1/1..Train loss: 2.773..Validation loss:  2.113..Validation accuracy: 0.483
Epoch 1/1..Train loss: 2.501..Validation loss:  1.850..Validation accuracy: 0.547
Epoch 1/1..Train loss: 2.648..Validation loss:  1.736..Validation accuracy: 0.570
Epoch 1/1..Tr

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [7]:
# TODO: Do validation on the test set
#TRAIN THE MODEL
epochs = 1
# Tracking the train steps
steps = 0
# Tracking the loss
running_loss = 0
# How many steps we're going to go before printing out the validation loss
print_every = 5

for epoch in range(epochs):
    # Training loop
    for images, labels in trainloader:
        steps += 1
        
        #move images and labels to GPU
        images, labels = images.to(device), labels.to(device)
        # We need to zero the gradients on each training pass or we'll retain gradients from previous training batches.
        optimizer.zero_grad()
        # Forward pass,  get our logits
        logps = model(images)
        # Calculate the loss with the logits and the labels
        loss = criterion(logps, labels)
        #then backward pass, then update weights
        loss.backward()
        # Take an update step and update the new weights
        optimizer.step()
        
        running_loss += loss.item()
        # Validation loop on the test data
        if steps % print_every == 0:
            # turn the model into evaluation mode (turn off dropout)
            model.eval()
            validation_loss = 0
            accuracy = 0
            # Turn off gradients for validation, saves memory and computations
            with torch.no_grad():
                for images, labels in testloader:
                
                    images, labels = images.to(device), labels.to(device)
                
                    logps = model(images)
                    loss = criterion(logps, labels)
                    validation_loss += loss.item()
                
                    # Calculate the accuracy
                    ps = torch.exp(logps)
                    top_ps, top_class = ps.topk(1, dim=1)
                    equality = top_class == labels.view(*top_class.shape)
                    # Calculate accuracy from equality
                    accuracy += torch.mean(equality.type(torch.FloatTensor))
             
            print(f"Epoch {epoch+1}/{epochs}.."
                  f"Train loss: {running_loss/print_every:.3f}.."
                  f"Validation loss: {validation_loss/len(testloader): .3f}.."
                  f"Validation accuracy: {accuracy/len(testloader):.3f}")
            
            running_loss = 0
            model.train()
        

Epoch 1/1..Train loss: 1.807..Validation loss:  1.273..Validation accuracy: 0.641
Epoch 1/1..Train loss: 1.945..Validation loss:  1.201..Validation accuracy: 0.681
Epoch 1/1..Train loss: 1.763..Validation loss:  1.255..Validation accuracy: 0.654
Epoch 1/1..Train loss: 1.861..Validation loss:  1.348..Validation accuracy: 0.660
Epoch 1/1..Train loss: 1.956..Validation loss:  1.250..Validation accuracy: 0.664
Epoch 1/1..Train loss: 1.711..Validation loss:  1.187..Validation accuracy: 0.700
Epoch 1/1..Train loss: 1.549..Validation loss:  1.198..Validation accuracy: 0.700
Epoch 1/1..Train loss: 1.575..Validation loss:  1.235..Validation accuracy: 0.686
Epoch 1/1..Train loss: 1.716..Validation loss:  1.141..Validation accuracy: 0.707
Epoch 1/1..Train loss: 1.922..Validation loss:  1.194..Validation accuracy: 0.678
Epoch 1/1..Train loss: 1.843..Validation loss:  1.175..Validation accuracy: 0.694
Epoch 1/1..Train loss: 1.812..Validation loss:  1.211..Validation accuracy: 0.686
Epoch 1/1..Train

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [8]:
# TODO: Save the checkpoint 
# The parameters for PyTorch networks are stored in a model's state_dict. 
# And the state dict contains the weight and bias matrices for each of the layers.
model.class_to_idx = image_datasets['train'].class_to_idx
checkpoint = {'arch': 'vgg11',
              'epochs': epochs,
              'state_dict': model.state_dict(),
              'class_to_idx': model.class_to_idx,
              'optimizer': optimizer.state_dict(),
              'input_size': 25088,
              'output_size': 102,
              'classifier' : classifier
             }


torch.save(checkpoint,'checkpoint.pth')
optimizer.state_dict

<bound method Optimizer.state_dict of Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.003
    weight_decay: 0
)>

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [9]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = models.vgg11(pretrained = True)
    model.classifier = checkpoint['classifier']
    model.load_state_dict = checkpoint['state_dict']
    model.class_to_idx = checkpoint['class_to_idx']
    
    
    return model

In [10]:
# define a checkpoint model by calling the load_checkpoint function
checkpoint_model = load_checkpoint('checkpoint.pth')
checkpoint_model.to(device);
print(checkpoint_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 512, kern

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [11]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    from PIL import Image
    im = Image.open(image)
    
    # resize the image
    image_size = im.size
    width = image_size[0]
    height = image_size[1]
    
    # im.thumbnail requires height and width to be tuple
    if width > height:
        im.thumbnail((5000000, 256)) #used the arbitrary 5000000, but any large value will work
    else:
        im.thumbnail((256, 5000000)) #used the arbitrary 5000000, but any large value will work
        
    # crop out the center 224x224 portion of the image
    left = (width - 224)/2
    top = (height - 224)/2
    right = (width + 224)/2
    bottom = (height + 224)/2

    im = im.crop((left, top, right, bottom))
    
    # Get Numpy array image from a PIL image
    np_image = np.array(im)
    
    # Normalization
    np_image = np_image/255
    mean = np.array([0.485, 0.456, 0.406])
    std = np.std([0.229, 0.224, 0.225])
    nor_image = (np_image - mean) / std
    
    # PyTorch tensors assume the color channel is the first dimension
    # turn a tensor of shape [width, height, color_channels] into [color_channels, width, hight]
    final_image = nor_image.transpose((2, 0, 1))
    
    #returns an Numpy array image
    return final_image
    

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [12]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [13]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    model.eval()
    image = process_image(image_path)
    # process_image function get a Numpy array image, we need to Convert Numpy array image to a PyTorch Tensor
    torch_image_tensor = torch.from_numpy(image).type(torch.FloatTensor).to('cpu')
    torch_image_tensor.unsqueeze_(0) # this needs to be added, to add a batch dimension to the input

    #torch_image_tensor = torch.tensor([image]).to('cpu')
    
    log_ps = model(torch_image_tensor)
                
    ps = torch.exp(log_ps)
    probs, labels = ps.topk(5, dim=1)
    
    # When training, we are not using the flower name or class number as labels.
    # We are using the index as labels in training/validation/testing. 
    # Hence, the output from prediction is also index and not flower name or class.
    # So, we need to get back the the actual flower name from the index.
    # For that, we do a reverse-lookup in the class_to_idx dictionary to get the class.
    
    idx_to_class = {val: key for key, val in model.class_to_idx.items()}
    
    classes = []
    for label in labels.numpy():
        classes.append(idx_to_class[label])
    
    return probs, classes

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [14]:
# TODO: Display an image along with the top 5 classes
from PIL import Image
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'
check_image = Image.open(test_dir + '/1/image_06743.jpg')
img_name = cat_to_name['1']

probs, classes = predict(test_dir + '/1/image_06743.jpg', checkpoint_model)

names = [cat_to_name[img_class] for img_class in classes]

plt.subplot(2, 1, 1)
plt.title(img_name)
plt.imshow(test_image)

plt.subplot(2, 1, 2)
plt.barh(names, probs)
plt.xlabel('Probabilities')

RuntimeError: Expected object of type torch.FloatTensor but found type torch.cuda.FloatTensor for argument #2 'weight'